<a href="https://colab.research.google.com/github/candle16/statcan-crime-analysis/blob/main/Crime_Analysis_Canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

# Configure Git with your info
!git config --global user.name "candle16"
!git config --global user.email "kateskipton@gmail.com"

# Store credentials so you don't have to enter token every time
!git config --global credential.helper store.

In [8]:
# Clone your repository
!git clone https://github.com/candle16/statcan-crime-analysis.git

# Navigate into the folder
%cd statcan-crime-analysis

# Check that you're in the right place
!pwd

Cloning into 'statcan-crime-analysis'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 1), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 9.04 KiB | 3.01 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/statcan-crime-analysis/statcan-crime-analysis
/content/statcan-crime-analysis/statcan-crime-analysis


In [9]:
!mkdir -p data
!mkdir -p outputs
!mkdir -p scripts

In [11]:
# CELL 1: Install Dependencies
!pip install stats-can openpyxl

In [ ]:
# CELL 2: Fetch Organized Crime Data from Statistics Canada
# Dataset: Table 35-10-0062-01 - Organized crime, by detailed violations
# URL: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3510006201

import pandas as pd
import requests
import zipfile
from io import BytesIO
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Create directories
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)
outputs_dir = Path('outputs')
outputs_dir.mkdir(exist_ok=True)

print("=" * 80)
print("FETCHING ORGANIZED CRIME DATA FROM STATISTICS CANADA")
print("=" * 80)

# StatCan table ID for organized crime
TABLE_ID = "35100062"

# Construct the download URL
download_url = f"https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid={TABLE_ID}&latestN=0&startDate=&endDate=&csvLocale=en&selectedMembers=%5B%5B%5D%2C%5B%5D%5D"

try:
    print(f"\n📥 Downloading data from Statistics Canada (Table {TABLE_ID})...")
    print(f"URL: {download_url[:80]}...")
    
    # Download the data
    response = requests.get(download_url, timeout=30)
    response.raise_for_status()
    
    # Extract ZIP file
    with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
        # Get the CSV file name (should be the table ID)
        csv_files = [f for f in zip_file.namelist() if f.endswith('.csv')]
        
        if not csv_files:
            raise ValueError("No CSV file found in downloaded ZIP")
        
        csv_filename = csv_files[0]
        print(f"✓ Downloaded and extracted: {csv_filename}")
        
        # Read the CSV
        with zip_file.open(csv_filename) as csv_file:
            df = pd.read_csv(csv_file)
    
    print(f"✓ Data loaded successfully: {len(df):,} rows, {len(df.columns)} columns")
    
    # Save raw data
    raw_data_path = data_dir / 'organized_crime_raw.csv'
    df.to_csv(raw_data_path, index=False)
    print(f"✓ Raw data saved to: {raw_data_path}")
    
    # Display basic info
    print(f"\n📊 Dataset Overview:")
    print(f"   • Time period: {df['REF_DATE'].min()} to {df['REF_DATE'].max()}")
    print(f"   • Columns: {', '.join(df.columns.tolist())}")
    
except requests.exceptions.RequestException as e:
    print(f"❌ Error downloading data: {e}")
    raise
except Exception as e:
    print(f"❌ Error processing data: {e}")
    raise

print("\n" + "=" * 80)
print("✓ DATA FETCH COMPLETE")
print("=" * 80)

In [ ]:
# CELL 3: Process and Clean Data
print("=" * 80)
print("PROCESSING ORGANIZED CRIME DATA")
print("=" * 80)

# Display the structure
print(f"\n📋 Data Structure:")
print(df.head())
print(f"\n📊 Column Types:")
print(df.dtypes)

# Clean and prepare the data
# Filter for actual crime violations (not totals in the violation name)
df_clean = df.copy()

# Convert REF_DATE to integer year
df_clean['Year'] = df_clean['REF_DATE'].astype(int)

# Filter for 2016-2024 only
df_clean = df_clean[df_clean['Year'].between(2016, 2024)]

# Get the value column (usually 'VALUE')
value_col = 'VALUE'
if value_col not in df_clean.columns:
    # Try to find the value column
    possible_cols = [col for col in df_clean.columns if 'value' in col.lower()]
    if possible_cols:
        value_col = possible_cols[0]
    else:
        raise ValueError("Could not find value column in data")

print(f"\n✓ Using '{value_col}' as the value column")

# Remove any rows with missing values
df_clean = df_clean[df_clean[value_col].notna()]

# Get violation column name
violation_col = 'Violations'
if violation_col not in df_clean.columns:
    possible_cols = [col for col in df_clean.columns if 'violation' in col.lower()]
    if possible_cols:
        violation_col = possible_cols[0]
    else:
        raise ValueError("Could not find violation column in data")

print(f"✓ Using '{violation_col}' as the violation column")

# Display unique violations
print(f"\n🔍 Unique violations found: {df_clean[violation_col].nunique()}")
print("\nSample violations:")
for i, violation in enumerate(df_clean[violation_col].unique()[:10], 1):
    print(f"   {i}. {violation}")

# Filter out "Total" rows to get specific violations
df_violations = df_clean[~df_clean[violation_col].str.contains('Total', case=False, na=False)]

print(f"\n✓ Filtered to {len(df_violations):,} rows of specific violations")
print(f"✓ Years covered: {df_violations['Year'].min()} to {df_violations['Year'].max()}")
print(f"✓ Number of specific violation types: {df_violations[violation_col].nunique()}")

print("\n" + "=" * 80)
print("✓ DATA PROCESSING COMPLETE")
print("=" * 80)

In [ ]:
# CELL 4: Create Line Graph - Organized Crime Trends (2016-2024)
print("=" * 80)
print("CREATING LINE GRAPH VISUALIZATION")
print("=" * 80)

# Calculate total organized crime per year
yearly_totals = df_violations.groupby('Year')[value_col].sum().reset_index()
yearly_totals.columns = ['Year', 'Total_Violations']

print(f"\n📊 Yearly Totals:")
print(yearly_totals)

# Find the top 3 most common organized crime violations (by total across all years)
top_violations = df_violations.groupby(violation_col)[value_col].sum().nlargest(3)
print(f"\n🔝 Top 3 Most Common Organized Crime Violations (2016-2024):")
for i, (violation, total) in enumerate(top_violations.items(), 1):
    print(f"   {i}. {violation}: {total:,} total violations")

# Prepare data for the top 3 violations over time
top_3_names = top_violations.index.tolist()
df_top3 = df_violations[df_violations[violation_col].isin(top_3_names)]

# Pivot data for easier plotting
df_pivot = df_top3.pivot_table(
    index='Year', 
    columns=violation_col, 
    values=value_col, 
    aggfunc='sum'
).fillna(0)

# Create the line graph
fig, ax = plt.subplots(figsize=(14, 8))

# Plot total organized crime
ax.plot(yearly_totals['Year'], yearly_totals['Total_Violations'], 
        marker='o', linewidth=3, markersize=8, 
        label='Total Organized Crime', color='#2E86AB', linestyle='-')

# Define colors for top 3 violations
colors = ['#A23B72', '#F18F01', '#C73E1D']
linestyles = ['--', '-.', ':']

# Plot top 3 violations
for i, violation in enumerate(top_3_names):
    if violation in df_pivot.columns:
        ax.plot(df_pivot.index, df_pivot[violation], 
                marker='s', linewidth=2.5, markersize=6,
                label=violation, color=colors[i], linestyle=linestyles[i])

# Formatting
ax.set_xlabel('Year', fontsize=14, fontweight='bold')
ax.set_ylabel('Number of Violations', fontsize=14, fontweight='bold')
ax.set_title('Organized Crime Trends in Canada (2016-2024)\nTotal Organized Crime and Top 3 Violation Types', 
             fontsize=16, fontweight='bold', pad=20)

# Format axes
ax.set_xticks(range(2016, 2025))
ax.grid(True, alpha=0.3, linestyle='--')
ax.legend(fontsize=11, loc='best', framealpha=0.9)

# Format y-axis with commas
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))

# Add subtle background
ax.set_facecolor('#F8F9FA')
fig.patch.set_facecolor('white')

plt.tight_layout()

# Save the figure
output_path = outputs_dir / 'organized_crime_trends_2016_2024.png'
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"\n✓ Graph saved to: {output_path}")

plt.show()

print("\n" + "=" * 80)
print("✓ LINE GRAPH COMPLETE")
print("=" * 80)

In [ ]:
# CELL 5: Create Table - Top 20 Crimes in 2024 with Growth Metrics
print("=" * 80)
print("CREATING TOP 20 CRIMES TABLE")
print("=" * 80)

# Get 2024 data
df_2024 = df_violations[df_violations['Year'] == 2024].copy()

# Get 2019 data
df_2019 = df_violations[df_violations['Year'] == 2019].copy()

# Get 2016 data
df_2016 = df_violations[df_violations['Year'] == 2016].copy()

# Calculate totals for each year by violation type
violations_2024 = df_2024.groupby(violation_col)[value_col].sum()
violations_2019 = df_2019.groupby(violation_col)[value_col].sum()
violations_2016 = df_2016.groupby(violation_col)[value_col].sum()

# Get top 20 crimes by 2024 violations
top_20_violations = violations_2024.nlargest(20)

# Create the summary table
summary_data = []

for violation in top_20_violations.index:
    count_2024 = violations_2024.get(violation, 0)
    count_2019 = violations_2019.get(violation, 0)
    count_2016 = violations_2016.get(violation, 0)
    
    # Calculate growth
    if count_2019 > 0:
        growth_2019_2024 = ((count_2024 - count_2019) / count_2019) * 100
    else:
        growth_2019_2024 = float('inf') if count_2024 > 0 else 0
    
    if count_2016 > 0:
        growth_2016_2024 = ((count_2024 - count_2016) / count_2016) * 100
    else:
        growth_2016_2024 = float('inf') if count_2024 > 0 else 0
    
    summary_data.append({
        'Rank': len(summary_data) + 1,
        'Violation Type': violation,
        '2024 Violations': int(count_2024),
        'Growth 2019-2024 (%)': growth_2019_2024,
        'Growth 2016-2024 (%)': growth_2016_2024
    })

# Create DataFrame
df_summary = pd.DataFrame(summary_data)

# Format the table for display
df_display = df_summary.copy()
df_display['2024 Violations'] = df_display['2024 Violations'].apply(lambda x: f'{x:,}')
df_display['Growth 2019-2024 (%)'] = df_display['Growth 2019-2024 (%)'].apply(
    lambda x: 'N/A' if x == float('inf') else f'{x:+.1f}%'
)
df_display['Growth 2016-2024 (%)'] = df_display['Growth 2016-2024 (%)'].apply(
    lambda x: 'N/A' if x == float('inf') else f'{x:+.1f}%'
)

print(f"\n📊 TOP 20 ORGANIZED CRIMES IN CANADA (2024)")
print("=" * 80)
print(df_display.to_string(index=False))
print("=" * 80)

# Save to CSV
csv_path = outputs_dir / 'top_20_crimes_2024_with_growth.csv'
df_summary.to_csv(csv_path, index=False)
print(f"\n✓ Table saved to: {csv_path}")

# Create a nicely formatted display table
from IPython.display import display, HTML

# Style the dataframe for better display
styled_df = df_display.style.set_properties(**{
    'text-align': 'left',
    'font-size': '12px',
    'border-color': 'black'
}).set_table_styles([
    {'selector': 'th', 'props': [
        ('background-color', '#2E86AB'),
        ('color', 'white'),
        ('font-weight', 'bold'),
        ('text-align', 'center'),
        ('padding', '10px')
    ]},
    {'selector': 'td', 'props': [
        ('padding', '8px'),
        ('border', '1px solid #ddd')
    ]},
    {'selector': 'tr:nth-of-type(even)', 'props': [
        ('background-color', '#f9f9f9')
    ]},
    {'selector': 'tr:hover', 'props': [
        ('background-color', '#f5f5f5')
    ]}
])

print("\n📋 FORMATTED TABLE:")
display(styled_df)

# Print summary statistics
print("\n" + "=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)
total_2024 = df_summary['2024 Violations'].sum()
avg_growth_2019 = df_summary[df_summary['Growth 2019-2024 (%)'] != float('inf')]['Growth 2019-2024 (%)'].mean()
avg_growth_2016 = df_summary[df_summary['Growth 2016-2024 (%)'] != float('inf')]['Growth 2016-2024 (%)'].mean()

print(f"Total violations in top 20 (2024): {total_2024:,}")
print(f"Average growth (2019-2024): {avg_growth_2019:+.1f}%")
print(f"Average growth (2016-2024): {avg_growth_2016:+.1f}%")

print("\n" + "=" * 80)
print("✓ TABLE GENERATION COMPLETE")
print("=" * 80)